# Imports and Setups

In [1]:
! pip install transformers

In [2]:
! pip install accelerate -U

In [3]:
! pip install simpletransformers

Lendo o Arquivo

In [4]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [5]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [6]:
perc= 5
i = 5

In [7]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/original/base_emoint_train.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [8]:
base_cru_1p.head()

,text,label
0,“Worry is a down payment on a problem you may ...,2
1,My roommate: it's okay that we can't spell bec...,0
2,No but that's so cute. Atsu was probably shy a...,1
3,Rooneys fucking untouchable isn't he? Been fuc...,0
4,it's pretty depressing when u hit pan on ur fa...,3


# Classification Model

In [9]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn


In [10]:
train_df = base_cru_1p.copy()
train_df.head()

,text,label
0,“Worry is a down payment on a problem you may ...,2
1,My roommate: it's okay that we can't spell bec...,0
2,No but that's so cute. Atsu was probably shy a...,1
3,Rooneys fucking untouchable isn't he? Been fuc...,0
4,it's pretty depressing when u hit pan on ur fa...,3


In [11]:
train_df.rename(columns= {'texto':'text', 'label': 'labels'}, inplace=True)
train_df.head()

,text,labels
0,“Worry is a down payment on a problem you may ...,2
1,My roommate: it's okay that we can't spell bec...,0
2,No but that's so cute. Atsu was probably shy a...,1
3,Rooneys fucking untouchable isn't he? Been fuc...,0
4,it's pretty depressing when u hit pan on ur fa...,3


In [12]:
n_labels = train_df['labels'].nunique()
n_labels

4

In [13]:
classifica = ClassificationModel('roberta', 'roberta-base', num_labels=n_labels, args={'num_train_epochs':4,
                                                                          'train_batch_size':32,
                                                                          'max_seq_length':64,
                                                                          #'silent':True,
                                                                        #  "fp16":False,
                                                                          'overwrite_output_dir': True})

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
classifica.train_model(train_df)

  0%|          | 0/3257 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/102 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/102 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/102 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/102 [00:00<?, ?it/s]

(408, 0.4797817032565089)

In [15]:
import pandas as pd

i=3



arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emoint/teste_algo/base_emoint_'+str(perc)+'_gptneokey'+str(i+1)+'.csv'
gptneo = pd.read_csv(arq, sep = ';', decimal = ',')




arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emoint/lambada/base_emoint_'+str(perc)+'_lambada'+str(i+1)+'.csv'
lambada = pd.read_csv(arq, sep = ';', decimal = ',')


In [16]:
lambada.head()

,texto,generated,new_label
0,Now that @user has snapchat back it's a consta...,Now that @user has been banned I want to know ...,0
1,Now that @user has snapchat back it's a consta...,"Now that @user has been banned, what if we tol...",0
2,Now that @user has snapchat back it's a consta...,"Now that @user has quit, will try again soon. ...",0
3,Now that @user has snapchat back it's a consta...,"Now that @user has changed, hope it comes back...",0
4,Now that @user has snapchat back it's a consta...,"Now that @user has shut down, maybe they would...",0


In [17]:
gptneo.head()

,texto,generated,new_label,prob_text
0,Kinda wished I watched mischievous kiss before...,ive been watching mischievous kiss and kinda ...,1,0.989131
1,Kinda wished I watched mischievous kiss before...,@user watched mischievous kiss me and I was ...,1,0.987535
2,Kinda wished I watched mischievous kiss before...,Watch me tease you and kiss you like I mean ...,1,0.985621
3,"her fingers slide along your thighs, caressing...",thighs caressing the + she leaned down. It w...,1,0.983900
4,Watching driven by food and @user going to Dev...,watch driven eating nihari makes @user makes...,1,0.983165


In [18]:
import numpy as np

def prob_softmax(predito):

  mx = np.max(predito, axis=-1, keepdims=True)
  numerator = np.exp(predito - mx)
  denominator = np.sum(numerator, axis=-1, keepdims=True)

  soft = numerator/denominator

  prob = max(soft)

  # classe = np.argmax(soft)

  return prob


In [19]:
preds = classifica.predict(lambada['generated'].to_list())[1]
lambada['preds'] = [preds[i] for i in range(preds.shape[0])]
lambada['prob_text'] = lambada['preds'].progress_apply(lambda x:prob_softmax(x))
lambada['novo_label'] = lambada['preds'].progress_apply(lambda x:np.argmax(x))
lambada['mislabel'] = (lambada['novo_label']!= lambada['new_label']).astype('int')

  0%|          | 0/814 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

100%|██████████| 814/814 [00:00<00:00, 135391.34it/s]


In [20]:
preds = classifica.predict(gptneo['generated'].to_list())[1]
gptneo['preds'] = [preds[i] for i in range(preds.shape[0])]
gptneo['prob_text'] = gptneo['preds'].progress_apply(lambda x:prob_softmax(x))
gptneo['novo_label'] = gptneo['preds'].progress_apply(lambda x:np.argmax(x))
gptneo['mislabel'] = (gptneo['novo_label']!= gptneo['new_label']).astype('int')

  0%|          | 0/815 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

100%|██████████| 815/815 [00:00<00:00, 100315.70it/s]


In [21]:
lambada.describe()

,new_label,prob_text,novo_label,mislabel
count,814.0,814.000000,814.000000,814.000000
mean,0.0,0.893061,1.323096,0.646192
std,0.0,0.144988,1.234013,0.478445
min,0.0,0.342008,0.000000,0.000000
25%,0.0,0.847839,0.000000,0.000000
50%,0.0,0.970759,1.000000,1.000000
75%,0.0,0.989731,3.000000,1.000000
max,0.0,0.994378,3.000000,1.000000


In [22]:
gptneo.describe()

,new_label,prob_text,novo_label,mislabel
count,815.000000,815.000000,815.000000,815.000000
mean,1.190184,0.943468,1.182822,0.105521
std,1.241938,0.112576,1.246991,0.307413
min,0.000000,0.366899,0.000000,0.000000
25%,0.000000,0.968527,0.000000,0.000000
50%,1.000000,0.989803,1.000000,0.000000
75%,3.000000,0.993263,3.000000,0.000000
max,3.000000,0.994462,3.000000,1.000000
